<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11n_640_Tray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os
import numpy as np, torch, random
import shutil

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [12]:
# --- imports
import torch, random, numpy as np

# --- xywh(正規化) 90°回転
def _rotate_xywh_norm(xywh, k):
    x, y, w, h = xywh.unbind(-1)
    k = int(k) % 4
    if k == 0:
        xn, yn, wn, hn = x, y, w, h
    elif k == 1:   # +90° CCW
        xn, yn, wn, hn = 1.0 - y, x,       h, w
    elif k == 2:   # 180°
        xn, yn, wn, hn = 1.0 - x, 1.0 - y, w, h
    else:          # 270° (= -90°)
        xn, yn, wn, hn = y, 1.0 - x,       h, w
    return torch.stack([xn, yn, wn, hn], dim=-1)

# --- xyxy(ピクセル) 90°回転
def _rotate_xyxy_pix(xyxy, W, H, k):
    b = xyxy.clone()
    k = int(k) % 4
    if k == 0:
        return b
    for _ in range(k):  # 90°ずつ回す（CCW）
        x1,y1,x2,y2 = b[:,0].clone(), b[:,1].clone(), b[:,2].clone(), b[:,3].clone()
        b[:,0], b[:,1] = W - y2, x1
        b[:,2], b[:,3] = W - y1, x2
        W, H = H, W
    return b

def rotate_batch_90deg_callback(trainer):
    b = getattr(trainer, "batch", None)
    if not b or "img" not in b or b["img"].ndim != 4:
        return

    # 適用率（epoch とともに 0.7→0.2）
    e = getattr(trainer, "epoch", 0)
    E = getattr(trainer, "epochs", 300)
    p = float(np.interp(e, [0, 0.5*E, E], [0.7, 0.4, 0.2]))
    trainer.rot_seen = getattr(trainer, "rot_seen", 0) + 1
    if random.random() > p:
        return

    k = 1 if random.random() < 0.7 else 3  # 90°:270° = 7:3
    imgs = b["img"]
    H, W = imgs.shape[-2], imgs.shape[-1]
    b["img"] = torch.rot90(imgs, k=k, dims=[-2, -1])

    # bboxes があれば安全に回す（形式自動判定）
    if b.get("bboxes", None) is not None and b["bboxes"].numel() > 0:
        boxes = b["bboxes"]
        # 値域で形式推定（<=1.5 なら正規化とみなす）
        if float(boxes.max()) <= 1.5:
            b["bboxes"] = _rotate_xywh_norm(boxes, k)
        else:
            # ピクセル座標。xyxyかxywhかを簡易判定
            # （w,h が正の場合 → xywh の可能性が高い）
            if "bbox_format" in b and b["bbox_format"] == "xyxy":
                b["bboxes"] = _rotate_xyxy_pix(boxes, W, H, k)
            else:
                # xywh ピクセル → いったん正規化→回転→ピクセルに戻す
                cx, cy, bw, bh = boxes.unbind(-1)
                xn = cx / W; yn = cy / H; wn = bw / W; hn = bh / H
                rot = _rotate_xywh_norm(torch.stack([xn,yn,wn,hn], dim=-1), k)
                rcx, rcy, rw, rh = rot.unbind(-1)
                b["bboxes"] = torch.stack([rcx*W, rcy*H, rw*W, rh*H], dim=-1)

    trainer.rot_applied = getattr(trainer, "rot_applied", 0) + 1

def log_rotate_ratio(trainer):
    if getattr(trainer, "rot_seen", 0):
        r = trainer.rot_applied / max(1, trainer.rot_seen)
        print(f"[epoch {trainer.epoch}] rotate90 applied ratio: {r:.2f}")
        trainer.rot_seen = 0; trainer.rot_applied = 0

def register_callbacks(model):
    # できれば前段で。失敗したら start に登録
    try:
        model.add_callback("on_preprocess_batch", rotate_batch_90deg_callback)
    except Exception:
        model.add_callback("on_train_batch_start", rotate_batch_90deg_callback)
    model.add_callback("on_train_epoch_end", log_rotate_ratio)


In [ ]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

In [ ]:
#!pip install -U torch torchvision torchaudio
!pip -q install ultralytics==8.3.177 --no-deps

In [ ]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

In [ ]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

In [13]:
# yolo学習用関数（callbacks= を train() に渡さない）
from ultralytics import YOLO
import os

def train_yolo(
    model_path,
    data_yaml,
    epochs=300,
    batch_size=16,
    img_size=1280,
    output_dir="runs_experiment",
    iou_nms=0.9,
    mosaic=0.15, mixup=0.05, copy_paste=0.6, close_mosaic=15,
    degrees=5.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.2,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.1, fliplr=0.5, flipud=0.0,
    lr0=0.004,
    optimizer="AdamW",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="yolo_project",
    **kwargs,
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # ← ここでコールバックを登録（train()には渡さない）
    # model.add_callback("on_train_batch_start", rotate_batch_90deg_callback)
    register_callbacks(model)  # 可視化に反映＆適用率ログ

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        optimizer=optimizer, lr0=lr0, cos_lr=cos_lr, warmup_epochs=warmup_epochs,

        mosaic=mosaic, mixup=mixup, copy_paste=copy_paste, close_mosaic=close_mosaic,
        degrees=degrees, shear=shear, perspective=perspective,
        translate=translate, scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing, fliplr=fliplr, flipud=flipud,

        iou=iou_nms,     # ← OK

        amp=amp, workers=workers,
        project=output_dir, name=project_name,
        **kwargs,
    )


In [ ]:
# ====== 実行パラメータ ======
model_path = "/content/drive/MyDrive/yolo11n.pt"  # or "/content/drive/MyDrive/last.pt"
data_yaml = "/content/dataset/data.yaml"
project_name = "yolo11n_640_learning"
output_dir = "runs_experiment"

train_params = {
    "model_path": model_path,
    "data_yaml": data_yaml,
    "epochs": 300,
    "batch_size": 64,
    "lr0": 0.0025,
    "img_size": 640,
    "output_dir": output_dir,
    "cache": "ram",
    "rect": False,
    "multi_scale": False,

    "iou_nms": 0.55,              # ← ここを使う（"iou" は削除）

    "mosaic": 0.0, "mixup": 0.0, "copy_paste": 0.0, "close_mosaic": 0,
    "degrees": 0.0, "shear": 0.0, "perspective": 0.0,
    "translate": 0.05, "scale": 0.2,   # ← 重複を整理
    "hsv_h": 0.015, "hsv_s": 0.4, "hsv_v": 0.2,
    "erasing": 0.0, "fliplr": 0.2, "flipud": 0.0,

    "optimizer": "AdamW",
    "cos_lr": True,
    "warmup_epochs": 3,
    "amp": True,
    "workers": 2,
    "project_name": project_name,
    "cls": 1.0,
}
train_yolo(**train_params)



In [ ]:
#output_dir = "runs_experiment"
best_model_path = f"{output_dir}/{project_name}/weights/best.pt"
last_model_path = f"{output_dir}/{project_name}/weights/last.pt"

from google.colab import files
import os

for model_path in [best_model_path, last_model_path]:
    if os.path.exists(model_path):
        print(f"Downloading: {model_path}")
        files.download(model_path)
    else:
        print(f"[ERROR] {os.path.basename(model_path)} が見つかりません: {model_path}")

